In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.autograd import Variable
import torch.nn.functional as F
from utils import listdir, image_to_tensor

In [2]:
class PredictedVoxelDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.list_category = sorted(os.listdir('predicted_EVC_conv5'))[14:16]
        path = []
        categories = []
        for i, c in enumerate(self.list_category):
            path.extend([os.path.join(self.root_dir, c, s) for s in os.listdir(os.path.join(self.root_dir, c))])
            nSample = len([s for s in os.listdir(os.path.join(self.root_dir, c))])
            for n in range(nSample):
                categories.append(i)
        self.path = path
        self.categories = categories
    def __len__(self):
        return len(self.path)
    def __getitem__(self, idx):
        #category = self.list_category[idx]
        voxels = np.load(self.path[idx])[0]
        sample = {'voxel': voxels, 'category': self.categories[idx]}
        if self.transform:
            sample = self.transform(sample)
        return sample

    
class VisualFeatureDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        if self.root_dir.split('.')[-1] == 'npy':
            temp = np.load(self.root_dir, allow_pickle=True)
            features = temp.item()
        else:
            features = torch.load(self.root_dir)
        
        categories = sorted(set(i.split('/')[0] for i in features.keys()))
        classes = {c: n for n, c in enumerate(categories)}
        self.images = [img for img in features.keys()]
        self.classes = {img: classes[img.split('/')[0]] for img in features.keys()}
        self.features = np.stack([f for f in features.values()])
        
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        img = self.images[idx]
        feature = self.features[idx, :]
        sample = {'feature': feature, 'category': self.classes[img]}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [3]:
class LinearClassifier(torch.nn.Module):
    def __init__(self, nVox, nClass):
        super(LinearClassifier, self).__init__()
        self.linear = torch.nn.Linear(nVox, nClass)
        
    def forward(self, x):
        x = self.linear(x)
        return x
    
class NonLinearClassifier(torch.nn.Module):
    def __init__(self, nVox, nClass):
        super(NonLinearClassifier, self).__init__()
        self.linear = torch.nn.Linear(nVox, nVox)
        self.linear2 = torch.nn.Linear(nVox, nClass)
        
    def forward(self, x):
        x = self.linear(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        return x    

In [4]:
feature_dataset = VisualFeatureDataset(root_dir='features_conv5_all.pth.npy')

batch_size = 64
n_iters = 10000
epochs = n_iters / (len(feature_dataset)/batch_size)
nFeature = 12544
nClass = len(set(feature_dataset.classes.values()))
lr_rate = 0.001


In [5]:
train_set, val_set = torch.utils.data.random_split(feature_dataset, [len(feature_dataset)-5000, 5000])
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=0)

linear_model = LinearClassifier(nFeature, nClass)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(linear_model.parameters(), lr=lr_rate)


In [6]:
iteration = 0
acc = []
for epoch in range(int(epochs)):
    for i, sample_batched in enumerate(train_loader):
        train_correct = 0
        train_total = 0
        voxels = Variable(torch.Tensor(sample_batched['feature'].float()))
        categories = Variable(sample_batched['category'])
        optimizer.zero_grad()
        outputs = linear_model(voxels)
        train_loss = criterion(outputs, categories)
        train_loss.backward()
        optimizer.step()
        
        # train loss?
        
        iteration+=1
        if iteration%200==0:
            
            _, predicted = torch.max(outputs.data, 1)
            if len(predicted) == len(categories):
                train_total += outputs.size(0)
                train_correct += (predicted == categories).sum()
                train_accuracy = 100 * train_correct.item() / train_total
            print("Train: Iteration: {}. Loss: {}. Accuracy: {}%".format(iteration, train_loss.item(), train_accuracy))
                 
            
            test_correct = 0
            test_total = 0
            for samples in test_loader:
                voxels = Variable(torch.Tensor(samples['feature'].float()))
                categories = Variable(samples['category'])
                outputs = linear_model(voxels)
                _, predicted = torch.max(outputs.data, 1)
                test_loss = criterion(outputs, categories)
                test_total += outputs.size(0)
                if len(predicted) == len(categories):
                    test_correct += (predicted == categories).sum()
                    test_accuracy = 100 * test_correct.item() / test_total
#                     acc.append(accuracy)
#                     if accuracy < np.mean(acc[-3:].float()):
                        
            print("Test: Iteration: {}. Loss: {}. Accuracy: {}%".format(iteration, test_loss.item(), test_accuracy))
                    

Train: Iteration: 200. Loss: 7.508553981781006. Accuracy: 0.0%
Test: Iteration: 200. Loss: 7.172121047973633. Accuracy: 1.58%
Train: Iteration: 400. Loss: 6.313730716705322. Accuracy: 9.375%
Test: Iteration: 400. Loss: 6.561035633087158. Accuracy: 4.62%
Train: Iteration: 600. Loss: 5.779475688934326. Accuracy: 18.75%
Test: Iteration: 600. Loss: 6.101731777191162. Accuracy: 8.22%
Train: Iteration: 800. Loss: 5.2257819175720215. Accuracy: 25.0%
Test: Iteration: 800. Loss: 5.4199628829956055. Accuracy: 12.0%
Train: Iteration: 1000. Loss: 4.238009452819824. Accuracy: 37.5%
Test: Iteration: 1000. Loss: 5.790889739990234. Accuracy: 15.62%
Train: Iteration: 1200. Loss: 3.8164777755737305. Accuracy: 40.625%
Test: Iteration: 1200. Loss: 5.8679914474487305. Accuracy: 19.18%
Train: Iteration: 1400. Loss: 3.187185525894165. Accuracy: 68.75%
Test: Iteration: 1400. Loss: 4.508225440979004. Accuracy: 22.0%
Train: Iteration: 1600. Loss: 3.18043851852417. Accuracy: 60.9375%
Test: Iteration: 1600. Loss:

In [ ]:
# for i_batch, sample_batched in enumerate(dataloader):
#     print(i_batch, sample_batched['voxel'].shape, sample_batched['category'])

batch_size = 2
n_iters = 80
epochs = n_iters / (len(voxel_dataset)/batch_size)
nVox = voxel_dataset[0]['voxel'].shape[0]
nClass = len(set(voxel_dataset.categories))
lr_rate = 0.2

In [ ]:
voxel_dataset = PredictedVoxelDataset(root_dir='./predictedvggall/subj001/LOC_conv5')
print(len(voxel_dataset))
print(voxel_dataset[0]['voxel'].shape)

In [ ]:
linear_model = LinearClassifier(nVox, nClass)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(linear_model.parameters(), lr=lr_rate)
#optimizer = torch.optim.Adam(linear_model.parameters(), lr=lr_rate)

In [ ]:
# # for i, sample_batched in enumerate(train_loader):
# #     print(i, sample_batched['voxel'].size(), sample_batched['category'])
# print(len(test_loader))
# for sample in test_loader:
#     print(sample)

In [ ]:
# train the linear model

iteration = 0
acc = []
for epoch in range(int(epochs)):
    for i, sample_batched in enumerate(train_loader):
        voxels = Variable(torch.Tensor(sample_batched['voxel'].float()))
        categories = Variable(sample_batched['category'])
        optimizer.zero_grad()
        outputs = linear_model(voxels)

        loss = criterion(outputs, categories)
        loss.backward()
        optimizer.step()
        
        # train loss?
        
        iteration+=1
        if iteration%2==0:
            correct = 0
            total = 0
            for samples in test_loader:
                voxels = Variable(torch.Tensor(samples['voxel'].float()))
                outputs = linear_model(voxels)
                _, predicted = torch.max(outputs.data, 1)
                total += outputs.size(0)
                if len(predicted) == len(categories):
                    correct += (predicted == categories).sum()
                    accuracy = 100 * correct.item() / total
#                     acc.append(accuracy)
#                     if accuracy < np.mean(acc[-3:].float()):
                        
                    print("Iteration: {}. Loss: {}. Accuracy: {}%".format(iteration, loss.item(), accuracy))
                    